# 穩定可以執行的tkinter錄影程式
# 可以開啟相機至少700秒以上而且視窗錄影的

In [1]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk, Image
import numpy as np
import face_recognition
import random
import time
import dlib

# 追蹤所使用的顏色
rectangleColor = (0,165,255) 

# 創建一個tracker，這邊只是for找Zonejia
trackers_dict = {'Zone-Jia': dlib.correlation_tracker(),
                'Barack': dlib.correlation_tracker(),
                'mom': dlib.correlation_tracker()}

# 當trackingFace[人名] = 1時，就代表此tracker正被追蹤
trackingFaces = {'Zone-Jia': 0,
                'Barack': 0,
                'mom': 0}

# tracking_scores
tracking_scores = {'Zone-Jia': 0,
                'Barack': 0,
                'mom': 0}


# 預先載入加分&扣分物的圖片
obj_width = 40

garbage_img = cv2.imread('apple.jpeg')
garbage_img = cv2.resize(garbage_img,(40,40),interpolation=cv2.INTER_CUBIC)
garbage_rows, garbage_cols, channels = garbage_img.shape

present_img = cv2.imread('bad.png')
present_img = cv2.resize(present_img,(40,40),interpolation=cv2.INTER_CUBIC)
present_rows, present_cols, channels = present_img.shape

print(garbage_rows, garbage_cols, channels)
print(present_rows, present_cols, channels)


40 40 3
40 40 3


In [2]:
root = tk.Tk()

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]
zonejia_image = face_recognition.load_image_file("zonejia_image.jpg")
zonejia_face_encoding = face_recognition.face_encodings(zonejia_image)[0]
mom_image = face_recognition.load_image_file("mom.jpeg")
mom_face_encoding = face_recognition.face_encodings(mom_image)[0]

# 當等於True才處理該Frame，False則忽略
process_this_frame = True

# 單純攝影機設定
width, height = 1280, 720
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
face_landmarks_list = []

# 可追蹤的人名
FACE_NAMES = ['Zone-Jia', 'Barack', 'mom']
# ------------------遊戲初始區----------------------
# 初始化 加分掉落物
init_y = 5
garbage_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
garbage_location_y = np.zeros(5, dtype=np.int32)*10
# 初始化 扣分掉落物
present_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
present_location_y = np.zeros(5, dtype=np.int32)*10

# 倒數計時出現的加分數字(位置)
bonus_position = (0,0) # (x, y)

# 每隔10毫秒掉落距離
garbage_speed = 15

# Zone-Jia的分數
zonejia_score = 0

# 其他人的合加分數
other_score = 0
# ------------------遊戲初始區----------------------

tStart = -1
tEnd = -1
finalCount = 0
def peace_close():
    global root, cap
#     root.quit()
    root.destroy() 
    cap.release()
    
root.bind('<Escape>', lambda e: peace_close)
lmain = tk.Label(root)
lmain.pack()

close_btn = tk.Button(root, text='點一下關閉', command=peace_close)
close_btn.pack()

def show_frame():
    # Initialize some variables
    global face_locations, face_encodings, face_names
    
    global process_this_frame
    global garbage_location_x, garbage_location_y
    global present_location_x, present_location_y

    global score, tStart, tEnd, bonus_position, finalCount
    global trackingFaces, trackers_dict, tracking_scores
    global other_score, zonejia_score
    
    _, frame = cap.read()
    frame = cv2.flip(frame, 1) # 將照片左右轉向一下
#     print('frame.shape:', frame)
    
    
    # 畫圖開始
#     small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)    
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    # 計時
    if tStart < 0:
        tStart = time.time()
    tEnd = time.time()
    finalSecond = int(tEnd - tStart)

#     print('目前時間：', finalSecond)
    
    # 繪畫倒數加分
    if finalSecond >= 10:
        if finalCount == (finalSecond-10):
            finalCount += 1
            bonus_position = random.randint(2, width-obj_width*2),random.randint(2, height-obj_width*2)
        if bonus_position != (0,0):
            font = cv2.FONT_HERSHEY_DUPLEX
            # 適合十位數顯示
            cv2.putText(frame, str(finalSecond), bonus_position, font, 1.0, (255, 0, 255))
            x1, y2 = bonus_position
            cv2.rectangle(frame, (x1, y2-obj_width//2), (x1+obj_width, y2), (255, 0, 255))
            
    # 使用dlib.tracker時就不在乎，是否要跳Frame處理
    for face_name in trackingFaces:
        if trackingFaces[face_name]:
    #         print('tracking中')
            # Update the tracker and request information about the
            # quality of the tracking update
            trackingQuality = trackers_dict[face_name].update(frame)

            # If the tracking quality is good enough, determine the
            # updated position of the tracked region and draw the
            # rectangle
            # trackingQuality越高，代表越嚴格的追蹤(太不像就會放棄追蹤)
            if trackingQuality >= 3.75:
    #             print('tracking到好目標，繼續tracking')
                tracked_position = trackers_dict[face_name].get_position()

                left = int(tracked_position.left())
                top = int(tracked_position.top())
                right = left + int(tracked_position.width())
                bottom = top + int(tracked_position.height())
                cv2.rectangle(frame, (left, top),
                                (right, bottom),
                                rectangleColor ,2)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, face_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

                # 計算tracking的碰撞分數

                # 因為掉落物的位置是以原本Frame為基準

                # 判斷是否撞到 加分的garbage
                for jjj in range(garbage_location_x.size):
                    rec_x1, rec_y1 = int(garbage_location_x[jjj]), int(garbage_location_y[jjj])
                    rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
                    # 判斷是否撞到第jjj個 加分的garbage
                    if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                        tracking_scores[face_name] += 1
                        garbage_location_y[jjj] = init_y
                        garbage_location_x[jjj] = random.randint(2, width-obj_width)

                # 判斷是否撞到 扣分的present
                for jjj in range(present_location_x.size):
                    rec_x1, rec_y1 = int(present_location_x[jjj]), int(present_location_y[jjj])
                    rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
                    # 判斷是否撞到第jjj個 扣分的present
                    if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                        tracking_scores[face_name] -= 1
                        present_location_y[jjj] = init_y
                        present_location_x[jjj] = random.randint(2, width-obj_width)
            else:
                # 放棄追蹤，結束tracking
    #             print('tracking不到好目標，設定tarckingFace=0')
                trackingFaces[face_name] = 0

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        
        face_names = []
        known_encodings = [obama_face_encoding, zonejia_face_encoding, mom_face_encoding]
        
    

        # 找出所有臉中是否有符合 '歐巴馬臉'或'宗嘉臉'的
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
#             face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        
#             for i, face_distance in enumerate(face_distances):
#                 print("The test image has a distance of {:.2} from known image #{}".format(face_distance, i))
#                 print("- With a normal cutoff of 0.6, would the test image match the known image? {}".format(face_distance < 0.6))
#                 print("- With a very strict cutoff of 0.5, would the test image match the known image? {}".format(face_distance < 0.5))
#                 print()

            match = face_recognition.compare_faces(known_encodings , face_encoding, 0.5)
            name = "Unknown"
#             print('name = @@@@@', match)
            if match[0]:
                name = "Barack"
            elif match[1]:
                name = 'Zone-Jia'
            elif match[2]:
                name = 'mom'
            face_names.append(name)
        
        # 超級嘗試Coding 起點

        # 判斷是否要執行dlib的tracker動作
        
        for FACE_NAME in FACE_NAMES:
            # 如果上一次有找到宗嘉，這次則沒有用face_recognition找到的話：
            if FACE_NAME in face_names:
                face_index = face_names.index(FACE_NAME)

                # face, and we determine this based on the largest
                # area of the found rectangle. First initialize the
                # required variables to 0
                maxArea = 0
                x = 0
                y = 0
                w = 0
                h = 0

                # Loop over all faces and check if the area for this
                # face is the largest so far
                # We need to convert it to int here because of the
                # requirement of the dlib tracker. If we omit the cast to
                # int here, you will get cast errors since the detector
                # returns numpy.int32 and the tracker requires an int
                (_top, _right, _bottom, _left) = face_locations[face_index]

                _top *= 2
                _right *= 2
                _bottom *= 2
                _left *= 2
                if (_bottom - _top) * (_right - _left) > maxArea:
                    maxArea = (_bottom - _top) * (_right - _left)
#                     print('maxArea:', maxArea)
                # If one or more faces are found, initialize the tracker
                # on the largest face in the picture
                if maxArea > 0:

                    # Initialize the tracker
                    trackers_dict[FACE_NAME].start_track(frame,
                                        dlib.rectangle(_left - 10,
                                                       _top - 20,
                                                       _right + 10,
                                                       _bottom+ 20))

                    # Set the indicator variable such that we know the
                    # tracker is tracking a region in the image
                    trackingFaces[FACE_NAME] = 1
                
    # 匡出 被face_recognitionx找到的臉部，除了'Zone-Jia'之外

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if name in FACE_NAMES:
            trackers_dict[name].start_track(frame,
                                    dlib.rectangle(2*left - 10,
                                                   2*top - 20,
                                                   2*right + 10,
                                                   2*bottom+ 20))
            continue
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
    #         top *= 4
    #         right *= 4
    #         bottom *= 4
    #         left *= 4
        top *= 2
        right *= 2
        bottom *= 2
        left *= 2
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        
    # 超級嘗試Coding 底部

    process_this_frame = not process_this_frame
    
    
    # 計算人臉辨識到的 分數(會忽略追蹤物'Zone-Jia'的碰撞得分)
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        if name in FACE_NAMES:
            trackers_dict[name].start_track(frame,
                                    dlib.rectangle(2*left - 10,
                                                   2*top - 20,
                                                   2*right + 10,
                                                   2*bottom+ 20))
            continue

        top *= 2
        right *= 2
        bottom *= 2
        left *= 2

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # 判斷是否撞到 加分的garbage
        for jjj in range(garbage_location_x.size):
            rec_x1, rec_y1 = int(garbage_location_x[jjj]), int(garbage_location_y[jjj])
            rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width

            #             A.x+A.W >= B.x ： 臉的右邊界大於等於 Sprite-B 的左邊界。
#             A.x <= B.x+B.W ： 臉的左邊界小於等於 Sprite-B 的右邊界。
#             A.y+A.H >= B.y ： Sprite-A 的下邊界大於等於 Sprite-B 的上邊界。
#             A.y <= B.y+B.H ： Sprite-A 的上邊界小於等於 Sprite-B 的下邊界。

            if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                other_score += 1
                # 被撞到的好garbage_location_x[jjj]
                garbage_location_y[jjj] = init_y
                garbage_location_x[jjj] = random.randint(2, width-obj_width)
        # 判斷是否撞到 扣分的present
        for jjj in range(present_location_x.size):
            rec_x1, rec_y1 = int(present_location_x[jjj]), int(present_location_y[jjj])
            rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width

#             A.x+A.W >= B.x ： 臉的右邊界大於等於 Sprite-B 的左邊界。
#             A.x <= B.x+B.W ： 臉的左邊界小於等於 Sprite-B 的右邊界。
#             A.y+A.H >= B.y ： Sprite-A 的下邊界大於等於 Sprite-B 的上邊界。
#             A.y <= B.y+B.H ： Sprite-A 的上邊界小於等於 Sprite-B 的下邊界。

            if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                other_score -= 1
                # 被撞到的壞present_location_x[jjj]
                present_location_y[jjj] = init_y
                present_location_x[jjj] = random.randint(2, width-obj_width)
                

    # 繪畫掉落物
    
    for jjj in range(garbage_location_x.size):
        rec_x1, rec_y1 = int(garbage_location_x[jjj]), int(garbage_location_y[jjj])
        rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
        # 畫圖片一樣30x30
        if rec_y2 >= height or rec_x2 >= width:
            garbage_location_x[jjj] = random.randint(2, width-obj_width)
            garbage_location_y[jjj] = init_y
        frame[rec_y1:rec_y2, rec_x1:rec_x2] = garbage_img
    
    for jjj in range(present_location_x.size):
        rec_x1, rec_y1 = int(present_location_x[jjj]), int(present_location_y[jjj])
        rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
        
        # 畫圖片一樣30x30
        if rec_y2 >= height or rec_x2 >= width:
            present_location_x[jjj] = random.randint(obj_width, width-obj_width)
            present_location_y[jjj] = init_y
        frame[rec_y1:rec_y2, rec_x1:rec_x2] = present_img
    
    # 繪畫 '分數顯示'
    cv2.putText(frame, 'Zone-Jia:'+str(tracking_scores['Zone-Jia']), (10, 45), cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0, 0, 255), 1)
    cv2.putText(frame, 'Barack:'+str(tracking_scores['Barack']), (10, 95), cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0, 0, 255), 1)
    cv2.putText(frame, 'mom:'+str(tracking_scores['mom']), (10, 135), cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0, 0, 255), 1)
    cv2.putText(frame, 'Other:'+str(other_score), (width-400, 45), cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0, 255, 0), 1)
    
    # 畫圖結束
    
    # 設定掉落物下一個Frame的掉落Y座標
    garbage_location_y += garbage_speed
    present_location_y += garbage_speed

    # 若掉落物卡到外部圖，重設為下個Frame的位置 
    garbage_location_x[garbage_location_x >= width-obj_width] = random.randint(5, width-obj_width)
    garbage_location_y[garbage_location_y >= height-obj_width-10] = init_y

    present_location_x[present_location_x >= width-obj_width] = random.randint(5, width-obj_width)
    present_location_y[present_location_y >= height-obj_width-10] = init_y

    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_frame)

show_frame()
root.mainloop()

cap.release()
print('和平結束')

和平結束


In [3]:
# 緊急停止方案
peace_close()

TclError: can't invoke "destroy" command:  application has been destroyed